# **과제2 : 챗봇 만들기2**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기2
    * Vector DB에 데이터 추가하기
    * Retriever, memory, LLM를 연결하기
    * 실행시 이력 DB 생성하고 기록하기
    * test

## **1.환경준비**

### (1) 라이브러리 Import

In [3]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [4]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key) 

sk-proj-PyYeTUrNRICof8XKMz4jT3BlbkFJij9YaaPRgfgaXGltWY8n


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [62]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키' 
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩 
    * 1일차에서 제공한 csv 파일의 구조를 그대로 이용
    * 에이블스쿨 홈페이지 FAQ 데이터 수집(https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056)
        * 모든 질문을 csv 형태로 저장
    * 데이터프레임으로 저장하기

In [5]:
import pandas as pd

df = pd.read_csv('aivleschool_qa.csv', encoding='utf-8')
df.tail()

,구분,QA
53,기타,AICE (기존 AIFB) 자격증이 있어야 유리한가요? 네. AICE (기존 AIF...
54,기타,AICE 시험관련 문의는 어디에 해야하나요? help@aice.study 으로 문의...
55,기타,국민취업지원제도를 신청하고 싶습니다. 국민취업지원제도 신청과 관련 사항은 거주지 관...
56,기타,국민취업지원제도를 안해도 교육 참여되나요? 국민취업제도를 신청하지 않아도 KT 에이...
57,기타,국민취업지원제도 관련 직업훈련탐색표가 필요합니다. 직업훈련탐색표는 고용센터 또는 국...


* 벡터 데이터베이스
    * 1일차 벡터 데이터베이스를 그대로 이용
        * Embedding 모델 : text-embedding-ada-002
        * DB 경로 : ./db



In [6]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./database", 
                    embedding_function = embeddings  
)

* 데이터 입력
    * 기존 입력을 모두 제거하고 추가 사항만 모두 입력
    * meta data로 '구분' 칼럼 값 추가하기

In [7]:
database.get()

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [8]:
text_list = df['QA'].tolist()
metadata_list = df['구분'].tolist()

metadata = [{'구분': category} for category in metadata_list]

documents = [Document(page_content=text, metadata=meta) for text, meta in zip(text_list, metadata)]

database.add_documents(documents)

['901e891c-f54e-47ea-812e-c187c691c18d',
 'b00e5826-4b47-4f5c-826d-ad1d76beae37',
 '86bf2bf8-9ad5-4176-988c-daa57a169687',
 '45429598-663c-4dab-90ea-aede4b6d9e63',
 '22a307fb-21d8-4f62-bcfb-4c29a89c3d38',
 '77132bf3-be53-4cfb-b1df-0b523e35d0f0',
 'a20acd6d-75f6-4ba2-bf17-69146f7a9dcd',
 'd801c9bd-a165-4b64-aa7d-db70929daa41',
 '27054527-2785-4c60-8c60-94f1ee2e8568',
 '348b9cb5-aa15-4438-bfc6-2ca8212ee927',
 'c522e5ba-164b-4d6f-ba2a-b70ce55e4b06',
 '8fcb2c68-9f19-4669-ba8b-3f3f39b10d84',
 'b4a56328-bd69-4a3d-a5a6-b4dfbd6bbb9a',
 '0a903ce7-3f93-4bf9-ba0b-b3cb5613327b',
 '509c57df-1ff0-401a-82c6-6716305523cd',
 'b9f716ab-e23d-45f4-b7de-b124f154907b',
 '3e26fae4-231c-457b-ae0a-fc7d0b5e65b5',
 '7ea05875-5f55-48bc-8c6b-a86cb1db74e2',
 'bddc6480-c5dc-4f32-8440-20241e17675b',
 '261a342f-7207-45b8-b0ff-7c7967292497',
 '5cbb1a45-7ed0-48f2-95c3-8780564685b9',
 '7c7a94f1-82bb-4cda-bc0a-7999093fcc62',
 '22584f2f-2ffe-437b-807e-a57d21997449',
 '40d70498-c99d-4c93-ae47-4e9c0e937b6a',
 '03b08edc-1658-

* 입력된 데이터 조회

In [9]:
dtt = database.get()
docu = pd.DataFrame(dtt)
docu[['metadatas','documents']]

,metadatas,documents
0,{'구분': '교육/수강'},타사 교육과 병행 할 수 있나요? KT 에이블스쿨은 풀타임(09:00~18:00)으...
1,{'구분': '교육/수강'},출결 기준이 어떻게 될까요? 1. 당일 소정훈련시간의 50퍼센트 미만을 수강한 경우...
2,{'구분': '교육/수강'},교육 수료 기준은 무엇인가요? KT 에이블스쿨은 고용노동부 K-Digital Tra...
3,{'구분': '모집/선발'},추가합격제도가 있나요? 현재 KT 에이블스쿨은 추가합격 제도를 운영하고 있지 않습니...
4,{'구분': '국민내일배움카드'},"국민내일배움카드가 있어야만 지원이 가능한가요? 네, 국민내일배움카드 보유자만 지원이..."
5,{'구분': '교육/수강'},대면 교육도 진행하나요? 교육 과정 중 STEP1과 STEP2는 대부분 비대면 교육...
6,{'구분': '교육/수강'},휴가를 사용할 수 있나요? K-Digital Training (K-DT) 규정상 월...
7,{'구분': '기타'},국민취업지원제도를 안해도 교육 참여되나요? 국민취업제도를 신청하지 않아도 KT 에이...
8,{'구분': '교육/수강'},컴퓨터공학 비전공자도 커리큘럼을 따라갈 수 있을까요? 비전공자도 SW개발에 관심과 ...
9,{'구분': '모집/선발'},아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 1...


## **3.RAG+LLM모델**

* 모델 : ConversationalRetrievalChain
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정
    * memory 사용
* 요구사항
    * 질문 history 관리를 위한 이력 저장 DB 생성
        * DB 명 : db_chatlog
        * 테이블 명 : history
            * id INTEGER PRIMARY KEY : 이렇게 설정하면 자동증가 값으로 채워짐
            * datetime TEXT : 질문시점 yyyy-mm-dd hh:mi:ss
            * query TEXT : 질문
            * sim1 REAL : 첫번째 문서의 유사도 점수
            * sim2 REAL : 두번째 문서의 유사도 점수
            * sim3 REAL : 세번째 문서의 유사도 점수
            * answer TEXT : 답변
        * 유사도 점수는 similarity_search_with_score 메서드를 이용해서 저장해야 함
        * 질문과 답변이 진행될 때마다 history 테이블에 데이터 입력

* 관리용 DB, 테이블 생성

* 모델 선언

* 모델 사용 및 이력 확인

In [10]:
chat = ChatOpenAI(model="gpt-3.5-turbo")
k=3
retriever = database.as_retriever(search_kwargs={"k": k})

memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
                                  return_messages=True)#대화가 시작될때 memory가 실행되어야 한다.

qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True,  output_key="answer")

path = './db_chatlog/chatlog.db'
conn = sqlite3.connect(path)

cursor = conn.cursor()

cursor.execute('''DROP TABLE IF EXISTS history''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS history (
    id INTEGER PRIMARY KEY,
    datetime TEXT NOT NULL,
    query TEXT NOT NULL,
    sim1 REAL NOT NULL,
    sim2 REAL NOT NULL,
    sim3 REAL NOT NULL,
    answer TEXT NOT NULL
)
''')

conn.commit()
conn.close()

while True:
    query = input('질문 > ')
    query = query.strip()
    print(f'질문 : {query}')
    print('-' * 20)
    if len(query) == 0:
        break
    result = qa({"question": query})
    print(f'답변 : {result["answer"]}')
    print('=' * 50)
    
    dt = datetime.now()
    dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    
    score = []
    result2 = database.similarity_search_with_score(query, k = k)
    for doc in result2:
        score.append(round(doc[1], 5))
        
    conn = sqlite3.connect(path)

    history_data = pd.DataFrame({'datetime': [dt],'query':[query],'sim1': [score[0]],'sim2': [score[1]],'sim3': [score[2]],'answer':[result["answer"]]})
    history_data.to_sql('history', conn, if_exists='append', index=False)
    
    d2 = pd.read_sql('SELECT * FROM history', conn)
    display(d2)
    print(memory.load_memory_variables({}))

    conn.close()
lod_df = d2

OperationalError: unable to open database file

In [21]:
query = "지원자격은??"   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시

[(Document(page_content='이전에 불합격했을 경우 이번에 다시 지원할수 있나요? 지원자격에 해당되는 경우 다시 지원이 가능합니다.', metadata={'구분': '모집/선발'}), 0.24874033113636443), (Document(page_content='직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다. 향후 취업여부를 확인할 예정입니다.', metadata={'구분': '모집/선발'}), 0.2984158636495033), (Document(page_content='사업자도 지원할 수 있나요?\n사업자는 자영업자로 분류됩니다. 자영업자의 경우 관할 고용센터에서 국민내일배움카드 발급 여부를 검토하며, 카드가 발급되어 교육을 수강하여도 훈련장려금은 지급되지 않습니다. 다만, 고용보험을 임의가입한 자영업자의 경우 훈련장려금이 지급됩니다.', metadata={'구분': '모집/선발'}), 0.3135772579672813)]
--------------------------------------------------
유사도 점수 : 0.24874, 문서 내용: 이전에 불합격했을 경우 이번에 다시 지원할수 있나요? 지원자격에 해당되는 경우 다시 지원이 가능합니다.
유사도 점수 : 0.29842, 문서 내용: 직장인도 지원할 수 있나요?
KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다. 향후 취업여부를 확인할 예정입니다.
유사도 점수 : 0.31358, 문서 내용: 사업자도 지원할 수 있나요?
사업자는 자영업자로 분류됩니다. 자영업자의 경우 관할 고용센터에서 국민내일배움카드 발급 여부를 검토하며, 카드가 발급되어 교육을 수강하여도 훈련장려금은 지급되지 않습니다. 다만, 고용보험을 임의가입한 자영업자의 경우 훈련장려금이 지급됩니다.


In [19]:
display(log_df)

,id,datetime,query,sim1,sim2,sim3,answer


In [36]:
# def process_query(query, k=3):
#     result = database.similarity_search_with_score(query, k=k)
#     sim1 = round(result[0][1], 5)
#     sim2 = round(result[1][1], 5)
#     sim3 = round(result[2][1], 5)
#     dt = datetime.now()    
#     dt = dt.strftime('%Y-%m-%d %H:%M:%S')
#     answer_result = qa({"question": query})    
#     answer = answer_result["answer"]    
#     return dt, query, sim1, sim2, sim3, answer
# conn = sqlite3.connect(path) 
# while True:    
#     query = input('질문 > ').strip()    
#     if not query:        break   
#     print(f'질문 : {query}')   
#     print('-' * 20)        
#     dt, query, sim1, sim2, sim3, answer = process_query(query)        
#     data = pd.DataFrame({'datetime': [dt],'query': [query],'sim1': [sim1],'sim2': [sim2],'sim3': [sim3],'answer': [answer]})        
#     data.to_sql('history', conn, if_exists='append', index=False)        
#     print(f'답변 : {answer}')    
#     print('=' * 50)
#     df = pd.read_sql('SELECT * FROM history', conn)
#     display(df)
#     conn.close()

질문 : 나는 몇살이야?
--------------------
답변 : 이에 대한 정보는 제가 알 수 없습니다.


,id,datetime,query,sim1,sim2,sim3,answer
0,1,2024-06-04 15:03:21,kt에이블 스쿨에 지원하고싶은데 어떤 조건들을 만족해야해?,0.23284,0.24214,0.26018,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준으로 재직자는 지원이 ..."
1,2,2024-06-04 15:03:55,내가 만약 취업상태라면 어떻게해야할까?,0.27547,0.29558,0.30972,"KT 에이블스쿨은 미취업자를 대상으로 하고 있어서, 현재 취업 상태일 경우에는 지원..."
2,3,2024-06-04 15:04:27,그러면 에이블스쿨에 학겨하기 전에 퇴사를 하면 참여가능할까?,0.21229,0.25994,0.26083,"KT 에이블스쿨은 미취업자를 대상으로 하고, 교육 시작일 기준으로 재직자는 지원이 ..."
3,4,2024-06-04 16:08:36,나는 26살이야,0.40889,0.42293,0.45180,"네, 해당 교육 과정은 34세 이하의 지원 대상자를 대상으로 합니다. 다만, 35세..."
4,5,2024-06-04 16:09:08,나는 몇살이야?,0.38818,0.39459,0.40453,이에 대한 정보는 제가 알 수 없습니다.


질문 : 나는 26살이야
--------------------


ProgrammingError: Cannot operate on a closed database.